In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from skimage.io import imread, imshow
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Activation, Flatten, Dense 
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator, array_to_img

### Loading images dataset


In [ ]:
img_height=224
img_width=224
batch_size=50
trainpath= ('E:/Dataset')


train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # set validation split
test_datagen = ImageDataGenerator(rescale=1 / 255.0)

print("The data is being split into training and validation set")

train_generator = train_datagen.flow_from_directory(
    trainpath,# This is the target directory
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training') # set as training data
print("----------------------------------------------------------------")

validation_generator = train_datagen.flow_from_directory(
    trainpath, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation') # set as validation data

test_generator = test_datagen.flow_from_directory(
    directory="E:/testset/",
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    shuffle=False,
    seed=42)

#define labels for training
class_decomposition = train_generator.class_indices
print('Number of classes after applying decomposition approach: ',class_decomposition)

#define labels for testing
org_classes = test_generator.class_indices
print('original classes: ',org_classes)


In [ ]:
#check the number of images in each class in the training dataset
No_images_per_class = []
Class_name = []

for i in os.listdir('E:/Dataset'):
    train_class = os.listdir(os.path.join('E:/Dataset', i))
    No_images_per_class.append(len(train_class))
    Class_name.append(i)
    print('Number of images in {} = {} \n'.format(i, len(train_class)))

In [ ]:
Class_name

In [ ]:
#view the augmented data
sample_x, sample_y = next(train_generator)
plt.figure(figsize=(12, 9))
for i in range(6):
    plt.subplot(2, 3, i+1)
    sample = array_to_img(sample_x[i])
    plt.axis('off')
    plt.grid(False)
    plt.imshow(sample)
plt.show()

### Building the model
### Loading the ImageNet pre-trained network

In [ ]:
# Load the pretrained model without the output of the last convolution block 
base_model = ResNet50(include_top=False, input_shape=(224, 224, 3), weights = 'imagenet')

# Flatten the output layer to 1 dimension
x = Flatten()(base_model.output)

# Add a fully connected layer with 2048 hidden units and ReLU activation
x = Dense(1024, activation='relu')(x)

# Adapt the final classification layer of an ImageNet pre-trained cNN model to the decomposed classes.
predictions = Dense(len(class_decomposition), activation='softmax')(x)

base_model = Model(inputs=base_model.input, outputs=predictions)

base_model.summary()    

### Training the model based on deep-tuning mode

In [ ]:
#define checkpoint
checkpoint = ModelCheckpoint(filepath='DeTraC_model.hdf5', 
                             monitor='val_accuracy',
                             save_best_only=True, 
                             mode='auto',
                             verbose=1)   

#early stopping
earlystop=EarlyStopping( monitor="val_accuracy", 
                        patience=10,  
                        mode="auto")

model_callbacks = [checkpoint, earlystop]


optimize = SGD(learning_rate=0.0001, decay=0.9 / 5, momentum=0.9, nesterov=True)
DeTraC_model = Model(inputs=base_model.input, outputs=predictions)

#compile model
DeTraC_model.compile(loss='mse', optimizer=optimize, metrics=['accuracy']) 
    
# train the model
history=DeTraC_model.fit(train_generator, 
                               steps_per_epoch=train_generator.n//train_generator.batch_size,
                               validation_data= validation_generator,
                               validation_steps=validation_generator.n//validation_generator.batch_size,
                               epochs=2, 
                               callbacks= model_callbacks, 
                               verbose=1, shuffle= True)

In [ ]:
# plot the training loss and accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))
plt.grid()

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Make a prediction on a test data

In [ ]:
x_test , y_test = [] , []
for i in range(test_generator.n//1):
    a , b = test_generator.next()
    x_test.extend(a) 
    y_test.extend(b)
y_test= np.array(y_test)
y_test.shape

In [ ]:
# Predict the output
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()

#make prediction
y_score=DeTraC_model.predict(test_generator,steps=STEP_SIZE_TEST,verbose=1) # array with the probability predictions made by the model
y_prediction=np.argmax(y_score,axis=1)  ## predicted_class_indices
#
#
# ground truth labels
# array with the labels of the test data
y_true = np.argmax(y_test, axis=1)


In [ ]:
y_prediction

### Class composition


In [ ]:
## Refine the final classification using error-correction critira

# determine parameter k; the number of classes in the class decomposition component.
k=2

correction_prediction=[]
for i in y_prediction:
      correction_prediction.append(i // k)
    
correction_prediction=np.array(correction_prediction)
correction_prediction

### Model evaluation

In [ ]:
#get confusion matrix
cm = confusion_matrix(y_true, correction_prediction)
print(cm)

#plot
fig, ax = plot_confusion_matrix(conf_mat=cm,  figsize=(6, 6),
                                colorbar=False,
                                show_absolute=True,
                                show_normed=False,
                                class_names=org_classes,cmap="Blues")
plt.show()


#get classification report
print(classification_report(y_true, correction_prediction, target_names=org_classes))
#
#
#

# compute sensitivity and specificity for multi classification 
res = []
for l in list(range(len(org_classes))):
    prec,recall,_,_ = precision_recall_fscore_support(np.array(y_true)==l,
                                       np.array(correction_prediction)==l,
                                              pos_label=True,average=None)
    res.append([l,recall[0],recall[1]])  

    df= pd.DataFrame(res,columns = ['class','sensitivity','specificity']) 
print(df)

#overall sensitivity and specificity
df[1:].mean(axis = 0)